In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
x = torch.rand(2, 20)
net(x)

tensor([[-0.1085, -0.1162,  0.0169, -0.1898,  0.1210, -0.0392,  0.0531,  0.0156,
         -0.0872,  0.1307],
        [-0.0160, -0.2263,  0.0042, -0.2222,  0.0786, -0.0607,  0.0090,  0.0820,
          0.0197,  0.0729]], grad_fn=<AddmmBackward>)

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
        
    def forward(self, x):
        return self.out(F.relu(self.hidden(x)))

In [3]:
net = MLP()
net(x)

tensor([[-0.0798,  0.1225,  0.1897, -0.1559,  0.0990, -0.1771, -0.1846,  0.0264,
          0.1829, -0.2376],
        [-0.0897,  0.1967,  0.1949, -0.2243,  0.1397, -0.1371, -0.2433, -0.0117,
          0.2495, -0.2535]], grad_fn=<AddmmBackward>)

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block
            
    def forward(self, x):
        for block in self._modules.values():
            x = block(x)
        return x

In [5]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(x)

tensor([[-0.2660,  0.0662,  0.1062,  0.2058, -0.1953, -0.1735,  0.3348,  0.1324,
          0.0255,  0.1750],
        [-0.2731,  0.1932, -0.0025,  0.3666, -0.2131, -0.2070,  0.2236,  0.2495,
         -0.0629,  0.2016]], grad_fn=<AddmmBackward>)

In [6]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
        
    def forward(self, x):
        x = self.linear(x)
        x = F.relu(torch.mm(x, self.rand_weight) + 1)
        x = self.linear(x)
        while x.abs().sum() > 1:
            x /= 2
        return x.sum()

In [7]:
net = FixedHiddenMLP()
net(x)

tensor(0.0210, grad_fn=<SumBackward0>)

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
        
    def forward(self, x):
        return self.linear(self.net(x))
    
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(x)

tensor(-0.2833, grad_fn=<SumBackward0>)